In [1]:
"""testing the multidim wav2vec model against the reference model"""
import torch
import numpy as np
from lightning.pytorch import seed_everything
import seisbench.data as sbd

from transformers.models.wav2vec2.modeling_wav2vec2 import _compute_mask_indices, _sample_negative_indices
from transformers import Wav2Vec2Config
from transformers import Wav2Vec2ForPreTraining as RefWav2Vec2ForPreTraining
from seisLM.model.multidim_wav2vec2 import MultiDimWav2Vec2ForPreTraining

data = sbd.STEAD()
waveforms = data.get_waveforms(1265656)
input_values = torch.Tensor(waveforms).unsqueeze(0)

2024-06-17 19:29:46,404 | seisbench | WARNING | Component order not specified, defaulting to 'ZNE'.


In [2]:
model_output = {}
# for model_name in MODEL_NAMES:
config = Wav2Vec2Config.from_pretrained(
  '/scicore/home/dokman0000/liu0003/projects/seisLM/seisLM/configs/pretrain/model_config_4xdownsample.json'
)

print(config)

for scale_logits_in_quantization in [False, True]:
  print('------------')
  config.scale_logits_in_quantization = scale_logits_in_quantization
  model = MultiDimWav2Vec2ForPreTraining(config)

  # compute masked indices
  batch_size, num_channels, raw_sequence_length = input_values.shape
  sequence_length = model._get_feat_extract_output_lengths(
    raw_sequence_length).item()

  seed_everything(0)
  mask_time_indices = _compute_mask_indices(
      shape=(batch_size, sequence_length), mask_prob=0.2, mask_length=2
  )
  sampled_negative_indices = _sample_negative_indices(
      features_shape=(batch_size, sequence_length),
      num_negatives=model.config.num_negatives,
      mask_time_indices=mask_time_indices,
  )
  mask_time_indices = torch.tensor(
    data=mask_time_indices, device=input_values.device, dtype=torch.long)
  sampled_negative_indices = torch.tensor(
    data=sampled_negative_indices, device=input_values.device,
    dtype=torch.long
  )

  with torch.no_grad():
    outputs = model(input_values, mask_time_indices=mask_time_indices,
                    sampled_negative_indices=sampled_negative_indices)

  print('codevector_complexity', outputs.codevector_perplexity)

Wav2Vec2Config {
  "activation_dropout": 0.0,
  "adapter_attn_dim": null,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    256,
    256
  ],
  "conv_kernel": [
    3,
    3
  ],
  "conv_stride": [
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.0,
  "feat_quantizer_dropout": 0.0,
  "final_dropout": 0.0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_dropout_prob": 0.0,
  "hidden_size": 240,
  "initializer_range": 0.02,
  "input_dim": 3,
  "inter

Seed set to 0


codevector_complexity tensor(152.5682)
------------


Seed set to 0


codevector_complexity tensor(591.2267)
